In [1]:
pip install notebook langchain langchain-community langchain-huggingface sentence-transformers faiss-cpu transformers pypdf torch accelerate

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import langchain
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_huggingface import HuggingFacePipeline

# Check for GPU (CUDA for NVIDIA, MPS for Mac, or CPU)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

print(f"✅ Libraries imported successfully.")
print(f"✅ Running on: {device.upper()}")

c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported successfully.
✅ Running on: CPU


In [9]:
# --- CELL 2: Advanced Semantic Ingestion ---

import re
from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 1. SETUP PATHS
DB_FAISS_PATH = 'vector_store/db_faiss'
DATA_PATH = 'data/company_policy.pdf' # <--- Ensure your PDF is here

# 2. LOAD & CLEAN DATA
print(f"Loading {DATA_PATH}...")
loader = PyPDFLoader(DATA_PATH)
raw_documents = loader.load()

# Function to clean PDF artifacts
def clean_text(text):
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace/newlines
    text = re.sub(r'\.\.\.', '', text) # Remove '...' artifacts
    return text.strip()

# Apply cleaning
for doc in raw_documents:
    doc.page_content = clean_text(doc.page_content)

print(f"Loaded {len(raw_documents)} pages and cleaned text.")

# 3. DEFINE EMBEDDINGS MODEL
# We need this BEFORE chunking because Semantic Splitter uses it to measure "meaning"
print("Loading Embedding Model...")
embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={'device': 'cpu'} # Change to 'cuda' if you have NVIDIA GPU
)

# 4. SEMANTIC CHUNKING (The "Pinecone Strategy")
# This splitter groups sentences that are semantically similar.
# It breaks the chunk only when the TOPIC changes.
print("Splitting text based on Semantic Meaning (this takes longer)...")
text_splitter = SemanticChunker(
    embeddings, 
    breakpoint_threshold_type="percentile" # Splits when difference is in top X%
)

texts = text_splitter.split_documents(raw_documents)
print(f"✅ Created {len(texts)} Meaningful Chunks.")

# 5. CREATE & SAVE VECTOR DB
print("Creating Vector Database...")
db = FAISS.from_documents(texts, embeddings)
db.save_local(DB_FAISS_PATH)
print(f"✅ Vector Database saved to '{DB_FAISS_PATH}'")

Loading data/company_policy.pdf...
Loaded 4 pages and cleaned text.
Loading Embedding Model...
Splitting text based on Semantic Meaning (this takes longer)...
✅ Created 8 Meaningful Chunks.
Creating Vector Database...
✅ Vector Database saved to 'vector_store/db_faiss'


In [10]:
# --- DEBUG CELL: Inspect New Semantic Chunks ---
print(f"Total Semantic Chunks: {len(texts)}")

print("\n--- CHUNK #1 (First Concept) ---")
print(texts[0].page_content)

print("\n--- CHUNK #2 (Next Concept) ---")
print(texts[1].page_content)

print("\n--- CHUNK #2 (Next Concept) ---")
print(texts[3].page_content)

Total Semantic Chunks: 8

--- CHUNK #1 (First Concept) ---
Company Policy Version 9 Confidentialy class public 09.04.2024 As a multinational company and therefore part of the overall global system, we see our daily behavior as a contribution to the implementation of the principle of sustainability. As a commercial enterprise, we acknowledge our special responsibility and want to make our contribution to preserving natural living conditions. Our company policy represents the foundation for this behavior. This corporate policy is relevant to all employees and contractors working for or on behalf of ST Extruded Products Group (STEP -G) and ST Germany (STD). All visitors are also requested to comply with this policy. Each of the above -mentioned groups is responsible for compliance with this policy at the locations in Bitterfeld (GER), Bonn (GER), Hettstedt (GER), Vogt (GER), Duffel (BEL) , Traun (AUT) and Tianjin (PRC). The fundamental basis are management systems in accordance with curre

In [11]:

# 1. IMPORTS
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_classic.memory import ConversationBufferMemory
from langchain_community.vectorstores import FAISS

# 2. LOAD LOCAL MODEL (Ollama)
# Make sure you have installed Ollama and ran 'ollama pull llama3' in your terminal
print("Loading Local Llama 3 via Ollama...")
llm = ChatOllama(model="llama3", temperature=0) # temp=0 for factual answers

# 3. SETUP RETRIEVER
# Load the Vector DB from Cell 2
db = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization=True)
retriever = db.as_retriever(search_kwargs={"k": 2}) # Read top 5 chunks

# 4. DEFINE PROMPT
template = """
Context:
{context}

Question: 
{question}

Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# 5. BUILD CHAINS
# Chain A: Simple QA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

# Chain B: Chat Memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

qa_chain_with_memory = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    chain_type="stuff",
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
    verbose=False
)

print("✅ Local Llama 3 (Ollama) Loaded & Chains Ready.")

Loading Local Llama 3 via Ollama...
✅ Local Llama 3 (Ollama) Loaded & Chains Ready.


In [12]:
# --- Simple Ollama Tester ---

from langchain_ollama import ChatOllama

try:
    print("⏳ Connecting to local Ollama...")
    
    # Initialize the model (Change "llama3" to "mistral" if you pulled that instead)
    llm = ChatOllama(model="llama3") 
    
    # Send a simple test query
    print("Sending test query...")
    response = llm.invoke("100 words about yourself")
    
    print(f"✅ Success! Response from AI: {response.content}")

except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 Troubleshooting Tip: Make sure the Ollama application is running in the background!")

⏳ Connecting to local Ollama...
Sending test query...
✅ Success! Response from AI: I'm an AI, a language model designed to assist and communicate with humans. I don't have personal experiences or emotions like humans do, but I'm always learning and improving my abilities. My primary function is to provide helpful and accurate information on a wide range of topics, from science and history to entertainment and culture. I can generate text based on prompts, answer questions, and even engage in conversation. While I don't have consciousness or self-awareness, I strive to be informative, engaging, and respectful in my interactions with users.


In [13]:
# --- CELL 5: Smart Orchestration (Intent & Confidence) ---

def get_similiarity_score(query):
    """
    Manually checks how close the best document is to the query.
    Lower score = Better match (Euclidean Distance).
    """
    docs_and_scores = db.similarity_search_with_score(query, k=2)
    if not docs_and_scores:
        return 100.0, None # High distance (bad match)
    
    score = docs_and_scores[0][1] # The FAISS distance score
    content = docs_and_scores[0][0].page_content
    return score, content

def smart_query(user_query):
    """
    The Master Function:
    1. Checks Intent (Chat vs Q&A)
    2. Checks Confidence (Is the document relevant?)
    3. Returns the answer
    """
    
    # 1. INTENT DETECTION (Simple Rule-Based for Speed)
    # In a real production app, we would use a small classifier model here.
    chat_keywords = ['hi', 'hello', 'hey', 'thanks', 'thank you', 'bye']
    if user_query.lower().strip() in chat_keywords:
        return {
            "answer": "Hello! I am your AI Assistant. Ask me anything about your documents.",
            "source": "Chat Logic (No Retrieval)",
            "confidence": "N/A"
        }

    # 2. CONFIDENCE CHECK
    # We define a "Threshold". If distance is > 1.0, it's probably irrelevant.
    # Note: Threshold depends on the model. For MiniLM, 1.0 - 1.3 is usually the cutoff.
    confidence_threshold = 1.5
    score, best_doc = get_similiarity_score(user_query)
    
    print(f"[DEBUG] Raw Distance Score: {score:.4f}") # For your learning

    if score > confidence_threshold:
        return {
            "answer": "I'm sorry, but I couldn't find enough information in the document to answer that confidently.",
            "source": "Confidence Guardrail (Refusal)",
            "confidence": f"Low (Score: {score:.2f})"
        }

    # 3. RAG EXECUTION (If we passed the checks)
    result = qa_chain.invoke({"query": user_query})
    
    return {
        "answer": result['result'],
        "source": result['source_documents'][0].page_content[:100] + "...",
        "confidence": f"High (Score: {score:.2f})"
    }

print("✅ Smart Orchestrator Ready.")

✅ Smart Orchestrator Ready.


In [23]:
# --- CELL 6: Testing the Guardrails ---

def run_test(question):
    print(f"User: {question}")
    response = smart_query(question)
    print(f"AI: {response['answer']}")
    print(f"   [Meta]: Source='{response['source']}', Confidence='{response['confidence']}'")
    print("-" * 50)

# Run different scenariosrun_test("Hello")                               # Scenario 1: Chat
run_test("quality management of the company")             # Scenario 2: Valid Question (Change this to match your PDF)
    # Scenario 3: Irrelevant Question

User: quality management of the company
[DEBUG] Raw Distance Score: 1.1778
AI: According to the provided Company Policy, the quality management of ST Extruded Products Germany GmbH focuses on:

* Ensuring customer requirements are met throughout the entire supply chain with all their specifications
* Error prevention and constant product quality

In other words, the company aims for a "Zero Defects" approach in its quality management.
   [Meta]: Source='Page 3 of 4 | ST Extruded Products Germany GmbH Company Policy | Version: 9 | Confidentiality class:...', Confidence='High (Score: 1.18)'
--------------------------------------------------


In [17]:
# --- CELL 9: Tuning for Better Answers ---
from langchain_classic.chains import ConversationalRetrievalChain
# 1. INCREASE CONTEXT (The most important fix)
# We change k=2 to k=5 so it reads more of the document.
retriever.search_kwargs['k'] = 5

# 2. IMPROVE THE PROMPT
# We tell the model to be verbose and detailed.
detailed_template = """You are a helpful AI assistant. Read the context below carefully.
The context might contain headers or metadata. Ignore them and focus on the main content.
Provide a comprehensive and detailed answer to the question.
If the context doesn't contain the answer, say "I don't know".

Context:
{context}

Question: 
{question}

Detailed Answer:"""

detailed_prompt = PromptTemplate.from_template(detailed_template)

# 3. UPDATE THE CHAINS
# We re-initialize the chains with these new settings.

# Standard Chain (for smart_query)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": detailed_prompt}
)

# Memory Chain (for chat)
qa_chain_with_memory = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    chain_type="stuff",
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": detailed_prompt}, # Apply new prompt here too
    verbose=False
)

print("✅ System Tuned: Context increased to 5 chunks & Prompt improved.")

✅ System Tuned: Context increased to 5 chunks & Prompt improved.
